In [1]:
import os
import re
import pywintypes
import win32file
import win32con
import datetime
import piexif
import subprocess
from PIL import Image
from PIL.ExifTags import TAGS
from hachoir.parser import createParser
from hachoir.metadata import extractMetadata

# ------------------ CONFIG ------------------
ROOT_DIR = r'C:\Users\kuste\Desktop\project\Arhiv slik'  # Update this path
IMAGE_EXTS = ['.jpg', '.jpeg', '.JPG', '.JPEG']         # Case-insensitive extensions
VIDEO_EXTS = ['.mp4', '.MP4', '.mov', '.MOV']
YEAR_PATTERN = re.compile(r'\b(20[0-2][0-9]|2030)\b')
EXIFTOOL_PATH = r'C:\ProgramData\chocolatey\bin\exiftool.exe'  # Chocolatey install path
# --------------------------------------------

# ---------- HELPER FUNCTIONS ----------
def is_image(file):
    return any(file.lower().endswith(ext) for ext in IMAGE_EXTS)

def is_video(file):
    return any(file.lower().endswith(ext) for ext in VIDEO_EXTS)

def is_text_file(file):
    return file.lower() == "date.txt"

def get_exif_date(image_path):
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
        if exif_data:
            for tag_id, value in exif_data.items():
                tag_name = TAGS.get(tag_id, tag_id)
                if tag_name == 'DateTimeOriginal':
                    return datetime.datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
    except Exception as e:
        print(f"⚠️ EXIF read error for {image_path}: {str(e)}")
    return None

def get_video_creation_date(path):
    try:
        parser = createParser(path)
        if not parser:
            return None
        with parser:
            metadata = extractMetadata(parser)
        if metadata and metadata.has("creation_date"):
            return metadata.get("creation_date")
    except Exception as e:
        print(f"⚠️ Video metadata read error for {path}: {str(e)}")
    return None

def parse_date_txt(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            line = f.readline().strip()
            line = line.replace('\u200e', '')  # Remove LTR characters
            return datetime.datetime.strptime(line, "%A, %d %B %Y")
    except Exception as e:
        print(f"❌ Error reading date.txt in {file_path}: {e}")
    return None

def extract_year_from_folder(name):
    match = YEAR_PATTERN.search(name)
    if match:
        return datetime.datetime(int(match.group()), 1, 1)  # Default to Jan 1 of year
    return None

def set_file_times(path, dt):
    try:
        win_time = pywintypes.Time(dt)
        handle = win32file.CreateFile(
            path,
            win32con.GENERIC_WRITE,
            win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE | win32con.FILE_SHARE_DELETE,
            None,
            win32con.OPEN_EXISTING,
            win32con.FILE_ATTRIBUTE_NORMAL,
            None
        )
        win32file.SetFileTime(handle, win_time, win_time, win_time)
        handle.Close()
        return True
    except Exception as e:
        print(f"❌ Failed to set filesystem times for {path}: {e}")
        return False

def update_image_exif_metadata(image_path, dt):
    """Completely updates all EXIF datetime fields in an image"""
    try:
        dt_str = dt.strftime("%Y:%m:%d %H:%M:%S")
        
        # Load existing EXIF or create new
        try:
            exif_dict = piexif.load(image_path)
        except:
            exif_dict = {"0th": {}, "Exif": {}, "GPS": {}, "1st": {}}
            
        # Update all relevant EXIF fields
        exif_dict["Exif"][piexif.ExifIFD.DateTimeOriginal] = dt_str
        exif_dict["Exif"][piexif.ExifIFD.DateTimeDigitized] = dt_str
        exif_dict["0th"][piexif.ImageIFD.DateTime] = dt_str
        
        # Additional fields some phones might check
        exif_dict["Exif"][36867] = dt_str  # DateTimeOriginal (alternate)
        exif_dict["Exif"][36868] = dt_str  # DateTimeDigitized (alternate)
            
        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, image_path)
        return True
    except Exception as e:
        print(f"❌ EXIF update failed for {image_path}: {e}")
        return False

def update_video_metadata(video_path, dt):
    """Updates video metadata using exiftool"""
    try:
        dt_str = dt.strftime("%Y:%m:%d %H:%M:%S")
        cmd = [
            EXIFTOOL_PATH,
            '-CreateDate=' + dt_str,
            '-ModifyDate=' + dt_str,
            '-TrackCreateDate=' + dt_str,
            '-MediaCreateDate=' + dt_str,
            '-overwrite_original',
            video_path
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, creationflags=subprocess.CREATE_NO_WINDOW)
        if result.returncode != 0:
            print(f"❌ ExifTool failed for {video_path}: {result.stderr}")
            return False
        return True
    except Exception as e:
        print(f"❌ Video metadata update failed for {video_path}: {e}")
        return False

# ---------- MAIN EXECUTION ----------
def main():
    folders_scanned = 0
    files_updated = 0

    # Verify ExifTool is accessible
    try:
        subprocess.run([EXIFTOOL_PATH, '-ver'], check=True, capture_output=True)
    except Exception as e:
        print(f"❌ Critical: ExifTool not found at {EXIFTOOL_PATH}")
        print("Install via Chocolatey: 'choco install exiftool'")
        return

    for dirpath, _, filenames in os.walk(ROOT_DIR):
        folders_scanned += 1
        print(f"\n📁 Processing folder: {os.path.basename(dirpath)}")
        
        txt_date = None
        image_dates = []
        video_dates = []

        # Step 1: Check for date.txt
        for file in filenames:
            if is_text_file(file):
                txt_date = parse_date_txt(os.path.join(dirpath, file))
                if txt_date:
                    print(f"📝 Found date.txt with date: {txt_date.date()}")
                break

        # Step 2: Collect media dates if no txt_date
        if not txt_date:
            for file in filenames:
                full_path = os.path.join(dirpath, file)
                if is_image(file):
                    dt = get_exif_date(full_path)
                    if dt:
                        image_dates.append((file, dt))
                elif is_video(file):
                    dt = get_video_creation_date(full_path)
                    if dt:
                        video_dates.append((file, dt))

        # Step 3: Determine fallback date
        fallback_date = None

        if txt_date:
            fallback_date = txt_date
        elif image_dates:
            fallback_date = sorted(image_dates, key=lambda x: x[1])[-1][1]  # Newest image date
        elif video_dates:
            fallback_date = sorted(video_dates, key=lambda x: x[1])[-1][1]  # Newest video date
        else:
            fallback_date = extract_year_from_folder(os.path.basename(dirpath))

        if not fallback_date:
            print("⚠️ No valid date found. Skipping folder.")
            continue

        print(f"ℹ️ Using reference date: {fallback_date.date()}")

        # Step 4: Process files
        for file in filenames:
            full_path = os.path.join(dirpath, file)

            if is_image(file):
                dt = txt_date or get_exif_date(full_path) or fallback_date
                if update_image_exif_metadata(full_path, dt) and set_file_times(full_path, dt):
                    print(f"✔️ Updated image: {file}")
                    files_updated += 1

            elif is_video(file):
                dt = txt_date or get_video_creation_date(full_path) or fallback_date
                if update_video_metadata(full_path, dt) and set_file_times(full_path, dt):
                    print(f"✔️ Updated video: {file}")
                    files_updated += 1

    print(f"\n✅ Done. Folders scanned: {folders_scanned}, Files updated: {files_updated}")

if __name__ == "__main__":
    main()


📁 Processing folder: Arhiv slik
⚠️ No valid date found. Skipping folder.

📁 Processing folder: !Prema gore kranjske
ℹ️ Using reference date: 2021-09-23
✔️ Updated image: IMG_20210920_111621.jpg
✔️ Updated image: IMG_20210920_112058.jpg
✔️ Updated image: IMG_20210921_172137.jpg
✔️ Updated image: IMG_20210921_173506.jpg
✔️ Updated image: IMG_20210921_175826.jpg
✔️ Updated image: IMG_20210923_130216.jpg
✔️ Updated image: IMG_20210923_204824-je uredil(-a)_.jpg
✔️ Updated image: IMG_20210923_204824.jpg
✔️ Updated image: IMG_20210923_204837-je uredil(-a)_.jpg
✔️ Updated image: IMG_20210923_204837.jpg
✔️ Updated image: received_3173426426226982.jpeg
✔️ Updated video: VID_20210922_180333~2.mp4

📁 Processing folder: Arhiv
ℹ️ Using reference date: 2024-01-06
✔️ Updated image: .trashed-1739827208-WallX_1349137_1080x1920.jpeg
✔️ Updated image: 03f53d3f919db3b017d5b66e19dece65.jpg
✔️ Updated image: 1119991.jpg
✔️ Updated image: 137374_original_2738x3000.jpg
✔️ Updated image: 1d14f37c8047758a46100c

c:\Users\kuste\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (108576768 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


ℹ️ Using reference date: 2025-03-08
❌ EXIF update failed for C:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\.trashed-1739529906-IMG20250115114406.jpg: Given data isn't JPEG.
❌ EXIF update failed for C:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\.trashed-1739827143-IMG20250117101022.jpg: Given data isn't JPEG.
✔️ Updated video: .trashed-1739827153-VID20250115121721.mp4
❌ EXIF update failed for C:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\.trashed-1739827212-IMG20250110174747.jpg: Given data isn't JPEG.
✔️ Updated image: 1000027965.jpg
❌ EXIF update failed for C:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\IMG20240827165242.jpg: Given data isn't JPEG.
❌ EXIF update failed for C:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\IMG20240830141336.jpg: Given data isn't JPEG.
✔️ Updated image: IMG20240906181825~2.jpg
❌ EXIF update failed for C:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\IMG20240907114439